### Streamlining Cheminformatics Workflows with [datamol](https://datamol.io) 1. Data manipulation, descriptors and clustering 

The [datamol](datamol.io) package from the M2D2 group provides set of functions that simplify some common Cheminformatics tasks.  The best part of this package is that it wraps the RDKit. All of the RDKit functionality you know and love is there, it's just easier to access.  In this notebook I'll highlight some of basic data manipulation capabilities available in datamol. 
- Transforming units
- Adding molecules and descriptors to Pandas dataframes
- Filtering dataframes based on descriptor values
- Clustering

Install the necessary Python libraries

In [1]:
import pandas as pd
import datamol as dm
from tqdm.auto import tqdm
from rdkit.Chem import rdDepictor
import mols2grid
from ipywidgets import interact
import ipywidgets as widgets

### A few preliminary settings
A couple of settings to get us started.  Let's make sure our chemical structures will look good and that floating point values aren't shown with a ridiculous number of decimal places. 

In [2]:
rdDepictor.SetPreferCoordGen(True)
pd.options.display.float_format = '{:,.2f}'.format

### Reading data
Read an input dataset.  There are a couple of things to notice here.  
- The input data doesn't have column headers, we use the "names" argument to **pd.read_csv** to specify column names. 
- We are using a space as a delimiter, this is specified in the **sep** argument.

In [3]:
#url = "https://raw.githubusercontent.com/PatWalters/yamc/main/data/HERG.smi"
url = "/Users/pwalters/software/yamc/data/HERG.smi"
df = pd.read_csv(url,sep=" ",names=["SMILES","Name","pIC50"])

In [4]:
df.head()

,SMILES,Name,pIC50
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43


### Transforming units
This data has the pIC50, or the log10 of the IC50 in $\mu$M.  Whenever I look at pIC50 values I end up mentally coverting them to IC50 values.  I've become pretty good at this but I'd prefer to have the IC50 values in the table. Fortunately, datamol has a set of convenience methods in the **dm.molar** package that do this conversion.  This isn't something that's hard to do, but it's nice to have a simple function available to handle the conversion. For more on **dm.molar** consult the docs.  
```python
help(dm.molar)
```

In [5]:
df['IC50'] = dm.molar.log_to_molar(df.pIC50,'uM')
df.head()

,SMILES,Name,pIC50,IC50
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03,0.01
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33,0.05
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07,8.48
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83,0.15
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43,0.37


### Adding a molecule to a dataframe
The datamol function **from_df** provides a convenient means of adding an RDKit molecule column to a dataframe.

In [6]:
df['mol'] = dm.from_df(df, smiles_column="SMILES")
df.head()

,SMILES,Name,pIC50,IC50,mol
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03,0.01,<rdkit.Chem.rdchem.Mol object at 0x28585c5f0>
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33,0.05,<rdkit.Chem.rdchem.Mol object at 0x28585c9e0>
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07,8.48,<rdkit.Chem.rdchem.Mol object at 0x28585c6d0>
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83,0.15,<rdkit.Chem.rdchem.Mol object at 0x28585c820>
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43,0.37,<rdkit.Chem.rdchem.Mol object at 0x28585cac0>


### Calculating simple 1D descriptors
The datamol package has a function **batch_compute_many_descriptors** that calculates many of the molecular descriptors supported by the RDKit.  When called with the default arguments, this function calculates 22 different descriptors. While this is useful, I usually only want to add a few descriptors to a dataframe.  We can do this by passing a dictionary to the **properties_fn** argument of **batch_compute_many_descriptors**.  The key for this dictionary is the name of the resulting dataframe column and the value is the the function that will be called to calculate the descriptor. This function should take an RDKit molecule as an argument. These functions don't necessarily have to be datamol or RDKit functions.  You can also add your own functions. For instance let's define a function to get the size of the largest ring in a molecule. 

In [7]:
def max_ring_size(mol):
    """Get the size of the largest ring in a molecule

    :param mol: input_molecule
    :return: size of the largest ring or 0 for an acyclic molecule
    """
    ri = mol.GetRingInfo()
    atom_rings = ri.AtomRings()
    if len(atom_rings) == 0:
        return 0
    else:
        return max([len(x) for x in ri.AtomRings()])

Now we can define the dictionary with the properties we want to calculate.

In [8]:
my_prop_dict = {
    "mw" : dm.descriptors.mw,
    "logp" : dm.descriptors.clogp,
    "hbd" : dm.descriptors.n_lipinski_hbd,
    "hba" : dm.descriptors.n_lipinski_hba,
    "max_ring_size" : max_ring_size
}

The datamol function **batch_compute_many_descriptors** returns a new dataframe with the computed descriptors. Note a couple of paramater settings. 
- add_properties=False - if this is set to True, all the supported descriptors are calculated
- progress=True - show a progress bar

In [9]:
prop_df = dm.descriptors.batch_compute_many_descriptors(df.mol,properties_fn=my_prop_dict,add_properties=False,
                                             progress=True)

  0%|          | 0/4042 [00:00<?, ?it/s]

Now we have a dataframe with the properties, but it doesn't have any of the information from our original dataframe.  We need to merge the two dataframes. 

In [10]:
prop_df

,mw,logp,hbd,hba,max_ring_size
0,440.18,4.63,1,5,6
1,379.17,3.68,1,5,6
2,324.15,2.52,1,5,6
3,454.28,5.09,0,6,6
4,382.14,4.89,0,4,7
...,...,...,...,...,...
4037,493.21,3.38,2,10,6
4038,480.18,3.16,2,10,6
4039,494.19,3.81,1,10,6
4040,494.19,3.55,2,10,6


Merge the descriptors into the main dataframe.

In [11]:
df = pd.concat([df,prop_df],axis=1)

### Filtering a dataframe based on descriptor values 
Let's filter the dataframe to only include molecules meeting the Lipinski Rule of 5 (Ro5) criteria.  I see a lot of people doing this using the Pandas bracket notation to do something like this.  
```python
df = df[df['mw'] <= 500]
df = df[df['logp'] <= 5]
df = df[df['hbd'] <= 5
df = df[df['hba'] <= 10
```
To me, this feels convoluted. When I try to do this, I usuually forget the syntax or screw something up.  I prefer using the pandas **query** function. 

In [12]:
df_ro5_ok = df.query("mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10")
len(df_ro5_ok)

3476

We could also create a column indicating whether a molecule passed the Ro5

In [13]:
df['ro5_ok'] = (df.mw <= 500) & (df.logp <=5) & (df.hbd <= 5) & (df.hba <= 10)
df.ro5_ok.sum()

3476

What if we want to count how many of the Ro5 criteria a molecule meets?  We could simply write a function to convert a boolean column to integers and calculate the sum of these integer columns. 

In [14]:
def bool_to_int(bool_in):
    return bool_in.values.astype(int)
bool_to_int(df.mw <= 500)

array([1, 1, 1, ..., 1, 1, 1])

Now we can use the function above to count how many of the Ro5 rules a molecule passes.  A molecule that passes all of the rules will have **ro5_count** set to 4.  As a check we can make sure the number of molecules passing all the rules is the same as what we calculated above. 

In [15]:
df['ro5_count'] = bool_to_int(df.mw <= 500) + bool_to_int(df.logp <=5) + bool_to_int(df.hbd <= 5) + bool_to_int(df.hba <= 10)
len(df.query("ro5_count == 4"))

3476

### Clustering
The datamol package provides a couple of useful functions for clustering.  The **cluster_mols** function provides a simple means of clustering based on the Butina alogorithm using the RDKit's Morgan fingerprints.  You can perform Butina clustering with the RDKit, but it takes a few more steps.  

When I look at the results of clustering, I like to see similar molecules aligned in a similar fashion.  Datamol has a another convenience function **auto_align_many**, that aligns molecules with same scaffold.  Note that you can do clustering and alignment in one step with **auto_align_many**.  However, I prefer to do this as two step process that gives me a bit more control over the clustering and the alignment. Both **cluster_mols** and **auto_align_many** have several options.  Check out the docs for more information. 

In [16]:
# cluster the molecules
cluster_list = dm.cluster_mols(df.mol)
# create a list to hold the cluster ids, which we will add to the dataframe
cluster_idx = [-1] * len(df)
for i,cluster in enumerate(tqdm(cluster_list[0])):
    # align the structures for each cluster using Bemis Murcko frameworks
    dm.align.auto_align_many([df.mol.values[x] for x in cluster],copy=False,partition_method='scaffold')
    # add the cluster id to cluster_idx
    for c in cluster:
        cluster_idx[c] = i
# add a column with cluster ids to the dataframe
df['cluster'] = cluster_idx

  0%|          | 0/3503 [00:00<?, ?it/s]

Datamol has a convenience function **dm.viz.to_image** that wraps **rdkit.Chem.Draw.MolsToGridImage**.  I don't tend to use either of those.  In 99% of cases, I prefer to us **mol2grid.display** to look at grids of chemical structures and data.  Here's a function that uses **mols2grid** and **iteract** to look at clusters one at a time.  Type a cluster id into the box to see the molecules from that cluster. You can also use the arrows on the right side of the box to scroll through the clusters.

In [17]:
@interact(x = widgets.IntText(description="Cluster"))
def show_cluster(x):
    return mols2grid.display(df.query(f"cluster == {x}"),mol_col="mol",subset=["img","Name","pIC50"],
                             use_coords=True, prerender=True, 
                             transform = {"pIC50" : lambda x: f"{x:.2f}"})

interactive(children=(IntText(value=0, description='Cluster'), Output()), _dom_classes=('widget-interact',))